In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze_etl
COMMENT 'Camada Bronze para armazenamento de dados brutos';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver_etl
COMMENT 'Camada Silver para armazenamento de dados tratados';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold_etl
COMMENT 'Camada Golf para armazenamento de dados prontos para uso';

In [0]:

def merge_delta_table(df_join, tabela_destino):
    """
    Realiza MERGE (upsert) de um DataFrame em uma tabela Delta Lake.
    Mantém a mesma lógica do código original fornecido pelo usuário.

    Parâmetros:
    -----------
    spark : SparkSession
        Sessão Spark ativa.
    df_join : DataFrame
        DataFrame com os dados a serem inseridos/atualizados.
    tabela_destino : str
        Nome completo da tabela Delta (ex: 'workspace.bronze.boi_gordo').

    Retorno:
    --------
    None
    """

    if df_join is not None and df_join.limit(1).count() > 0:
        total = df_join.count()
        print(f"✅ Total de registros carregados: {total}")

        if spark.catalog.tableExists(tabela_destino):
            print(f"📦 Tabela {tabela_destino} já existe — atualizando dados...")

            delta_table = DeltaTable.forName(spark, tabela_destino)

            (
                delta_table.alias("t")
                .merge(
                    df_join.alias("s"),
                    "t.data_coleta = s.data_coleta AND t.data = s.data"
                )
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute()
            )

            print(f"✅ MERGE concluído com sucesso em {tabela_destino}")

        else:
            print(f"🆕 Tabela {tabela_destino} não existe — criando nova tabela...")

            df_join.write \
                .format("delta") \
                .partitionBy("data_coleta") \
                .mode("append") \
                .saveAsTable(tabela_destino)

            print(f"✅ Tabela {tabela_destino} criada e dados inseridos com sucesso.")

    else:
        print("⚠️ Nenhum dado para atualizar.")
